In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from dash_website.datasets import SEX_VALUE, ETHNICITIES, SEX_COLOR

scalars = pd.read_feather("../../all_data/datasets/scalars/PhysicalActivity_FullWeek_Scalars.feather").set_index(["sex", "id"])
selected_feature = "acc-hourOfWeekday-0-avg_V1"
min_age = max(scalars["chronological_age"].min(), 10)
max_age = min(scalars["chronological_age"].max(), 80)

In [2]:
scalars.drop(index=scalars[(scalars["chronological_age"] > max_age) | (scalars["chronological_age"] < min_age)].index, inplace=True)

In [3]:
scalars

chronological_age  acc-hourOfDay-0-avg_V1  \
sex id                                                        
1.0 3903485_1.5           48.887062                    9.05   
0.0 4308985_1.5           63.381245                    2.24   
1.0 2716336_1.5           70.718690                   11.97   
0.0 5729610_1.5           75.069130                    2.32   
    3605976_1.52          71.203285                    3.05   
...                             ...                     ...   
1.0 2031659_1.5           50.680355                    2.45   
    5847860_1.5           47.526352                   22.17   
0.0 4475598_1.5           60.982887                    3.75   
1.0 1806842_1.5           70.299800                    2.33   
0.0 4436173_1.5           60.605064                    2.55   

                  acc-hourOfWeekday-0-avg_V1  acc-hourOfWeekend-0-avg_V1  \
sex id                                                                     
1.0 3903485_1.5                         5.06                       19.01   
0.0 4308985_1.5                         2.28                        2.14   
1.0 2716336_1.5                        12.94                        9.57   
0.0 5729610_1.5                         2.47                        1.93   
    3605976_1.52                        2.70                        3.93   
...                                      ...                         ...   
1.0 2031659_1.5                         2.41                        2.54   
    5847860_1.5                        20.82                       25.56   
0.0 4475598_1.5                         3.84                        3.53   
1.0 1806842_1.5                         2.75                        1.27   
0.0 4436173_1.5                         2.75                        2.05   

                  acc-hourOfDay-10-avg_V1  acc-hourOfWeekday-10-avg_V1  \
sex id                                                                   
1.0 3903485_1.5                     27.14                        28.07   
0.0 4308985_1.5                     50.59                        30.20   
1.0 2716336_1.5                     64.27                        61.86   
0.0 5729610_1.5                     44.56                        47.45   
    3605976_1.52                    55.38                        50.08   
...                                   ...                          ...   
1.0 2031659_1.5                     28.10                        25.83   
    5847860_1.5                     41.49                        34.00   
0.0 4475598_1.5                     51.43                        54.53   
1.0 1806842_1.5                     44.97                        49.86   
0.0 4436173_1.5                     55.32                        51.73   

                  acc-hourOfWeekend-10-avg_V1  acc-hourOfDay-20-avg_V1  \
sex id                                                                   
1.0 3903485_1.5                         24.80                    43.34   
0.0 4308985_1.5                        101.56                    28.01   
1.0 2716336_1.5                         70.31                    24.62   
0.0 5729610_1.5                         37.36                    45.64   
    3605976_1.52                        68.64                    25.01   
...                                       ...                      ...   
1.0 2031659_1.5                         33.79                    32.87   
    5847860_1.5                         60.20                    44.44   
0.0 4475598_1.5                         43.69                    30.86   
1.0 1806842_1.5                         32.75                     5.46   
0.0 4436173_1.5                         64.29                    32.10   

                  acc-hourOfWeekday-20-avg_V1  acc-hourOfWeekend-20-avg_V1  \
sex id                                                                       
1.0 3903485_1.5                         40.05                        51.56   
0.0 4308985_1.5                         31.10    

In [4]:
import plotly.graph_objs as go

fig = go.Figure()

fig.add_histogram(x=scalars.loc[SEX_VALUE["female"], selected_feature], name="Females", histnorm="percent", marker=dict(color="pink"))

fig.add_histogram(x=scalars.loc[SEX_VALUE["male"], selected_feature], name="Males", histnorm="percent", marker=dict(color="blue"))

fig.update_layout(
    xaxis_title_text='Value',
    yaxis_title_text='Count (in %)',
    bargap=0.2,
    bargroupgap=0.1
)

fig.show()

In [5]:
list_indexes = []

for sex in ["all", "male", "female"]:
    for observation in ["slope", "intercept", "p_value", "correlation", "sample_size"]:
        list_indexes.append([sex, observation])
indexes = pd.MultiIndex.from_tuples(list_indexes, names=["sex", "observation"])
linear_regression = pd.DataFrame(None, index=indexes, columns=scalars.columns.drop(["chronological_age"] + ETHNICITIES))

In [6]:
from sklearn.linear_model import LinearRegression


for feature in scalars.columns.drop(["chronological_age"] + ETHNICITIES):

    linear_regressor = LinearRegression()
    linear_regressor.fit(scalars.reset_index(level="sex")[["sex"] + ETHNICITIES], scalars["chronological_age"])
    corrected_chronological_age = scalars["chronological_age"] - linear_regressor.predict(scalars.reset_index(level="sex")[["sex"] + ETHNICITIES])

    linear_regressor = LinearRegression()
    linear_regressor.fit(scalars.reset_index(level="sex")[["sex"] + ETHNICITIES], scalars[feature])
    corrected_feature = scalars[feature] - linear_regressor.predict(scalars.reset_index(level="sex")[["sex"] + ETHNICITIES])

    linear_regression.loc[("all", "correlation"), feature] = corrected_feature.corr(corrected_chronological_age, method="pearson")

    for sex in ["male", "female"]:
        linear_regressor_sex = LinearRegression()
        linear_regressor_sex.fit(scalars.loc[SEX_VALUE[sex], ETHNICITIES], scalars.loc[SEX_VALUE[sex], "chronological_age"])
        corrected_chronological_age_sex = scalars.loc[SEX_VALUE[sex], "chronological_age"] - linear_regressor_sex.predict(scalars.loc[SEX_VALUE[sex], ETHNICITIES])

        linear_regressor_sex = LinearRegression()
        linear_regressor_sex.fit(scalars.loc[SEX_VALUE[sex], ETHNICITIES], scalars.loc[SEX_VALUE[sex], feature])
        corrected_feature_sex = scalars.loc[SEX_VALUE[sex], feature] - linear_regressor_sex.predict(scalars.loc[SEX_VALUE[sex], ETHNICITIES])

        linear_regression.loc[(sex, "correlation"), feature] = corrected_feature_sex.corr(corrected_chronological_age_sex, method="pearson")


    slope, intercept, _, p_value, _ = linregress(scalars["chronological_age"], scalars[feature])
    
    linear_regression.loc[("all", "slope"), feature] = slope
    linear_regression.loc[("all", "intercept"), feature] = intercept
    linear_regression.loc[("all", "p_value"), feature] = p_value
    linear_regression.loc[("all", "sample_size"), feature] = scalars.shape[0]

    for sex in ["male", "female"]:
        slope_sex, intercept_sex, _, p_value_sex, _ = linregress(scalars.loc[SEX_VALUE[sex], "chronological_age"], scalars.loc[SEX_VALUE[sex], feature])

        linear_regression.loc[(sex, "slope"), feature] = slope_sex
        linear_regression.loc[(sex, "intercept"), feature] = intercept_sex
        linear_regression.loc[(sex, "p_value"), feature] = p_value_sex
        linear_regression.loc[(sex, "sample_size"), feature] = scalars.loc[SEX_VALUE[sex]].shape[0]


In [6]:
from scipy.stats import linregress
import numpy as np


fig = go.Figure()

fig.add_scattergl(y=scalars[selected_feature], x=scalars["chronological_age"], name=selected_feature, opacity=0.8, marker={"color":"Grey", "size":2}, mode="markers")



fig.add_scatter(y=linear_regression.loc[("all", "slope"), selected_feature] * np.array([min_age, max_age]) + linear_regression.loc[("all", "intercept"), selected_feature], x=[min_age, max_age], name="All", line={"color":"Black"}, marker={"size":0.1})

for sex in ["male", "female"]:
    fig.add_scatter(y=linear_regression.loc[(sex, "slope"), selected_feature] * np.array([min_age, max_age]) + linear_regression.loc[(sex, "intercept"), selected_feature], x=[min_age, max_age], name=sex.capitalize(), line={"color":SEX_COLOR[sex]}, marker={"size":0.1})


fig.update_layout(
    xaxis_title_text='Chronological Age',
)

fig.show()

In [7]:
linear_regression

Minimum carotid IMT (intima-medial thickness) at 120 degrees   \
sex    observation                                                                 
all    slope                                                 5.171812              
       intercept                                           253.891755              
       p_value                                                    0.0              
       correlation                                           0.300061              
       sample_size                                                523              
male   slope                                                  4.48903              
       intercept                                           311.684993              
       p_value                                               0.000096              
       correlation                                           0.227029              
       sample_size                                                253              
female slope                                                 5.585985              
       intercept                                            214.81524              
       p_value                                                    0.0              
       correlation                                            0.38403              
       sample_size                                                270              

                   Mean carotid IMT (intima-medial thickness) at 120 degrees   \
sex    observation                                                              
all    slope                                                 6.200785           
       intercept                                           286.370808           
       p_value                                                    0.0           
       correlation                                           0.338321           
       sample_size                                                523           
male   slope                                                 4.447613           
       intercept                                           419.267002           
       p_value                                               0.000227           
       correlation                                            0.21739           
       sample_size                                                253           
female slope                                                 7.517925           
       intercept                                            184.35364           
       p_value                                                    0.0           
       correlation                                           0.466238           
       sample_size                                                270           

                   Maximum carotid IMT (intima-medial thickness) at 120 degrees   \
sex    observation                                                                 
all    slope                                                 7.036265              
       intercept                                           343.611073              
       p_value                                                    0.0              
       correlation                                           0.307684              
       sample_size                                                523              
male   slope                                                 4.118576              
       intercept                                           563.757805              
       p_value                                               0.006135              
       correlation                                           0.165587              
       sample_size                                                253              
female slope                                                 9.245047              
       intercept                                           173.719636              
       p_value                      

In [20]:
-np.log(0.05 / linear_regression.loc[("all", "sample_size")][0])

9.255313737618915

In [26]:
fig = go.Figure()

hovertemplate = "Feature : %{customdata[0]}\
                    <br>p_value : %{customdata[1]:.3f}\
                    <br>Correlation : %{x:.3f}\
                    <br>Sample Size : %{customdata[2]}\
                    <br>Slope : %{customdata[3]:.3f}"
customdata = np.stack(
    [
        linear_regression.columns,
        linear_regression.loc[("all", "p_value")],
        linear_regression.loc[("all", "sample_size")],
        linear_regression.loc[("all", "slope")]
    ],
    axis=-1,
)

fig.add_scatter(x=linear_regression.loc[("all", "correlation")], 
                y=-np.log10((linear_regression.loc[("all", "p_value")] + 1e-16).to_list()),
                mode="markers",
                name="all",
                hovertemplate=hovertemplate,
                customdata=customdata,
                marker={"color":"Black"},
                )

for sex in ["male", "female"]:
    hovertemplate_sex = "Feature : %{customdata[0]}\
                        <br>p_value : %{customdata[1]:.3f}\
                        <br>Correlation : %{x:.3f}\
                        <br>Sample Size : %{customdata[2]}\
                        <br>Slope : %{customdata[3]:.3f}"
    customdata_sex = np.stack(
        [
            linear_regression.columns,
            linear_regression.loc[(sex, "p_value")],
            linear_regression.loc[(sex, "sample_size")],
            linear_regression.loc[(sex, "slope")]
        ],
        axis=-1,
    )

    fig.add_scatter(x=linear_regression.loc[(sex, "correlation")], 
                    y=-np.log10((linear_regression.loc[(sex, "p_value")] + 1e-16).to_list()),
                    mode="markers",
                    name=sex,
                    hovertemplate=hovertemplate_sex,
                    customdata=customdata_sex,
                    marker={"color":SEX_COLOR[sex]}
                    )

fig.add_scatter(
        x=[
            linear_regression.loc[("all", "correlation")].min() - linear_regression.loc[("all", "correlation")].std(),
            linear_regression.loc[("all", "correlation")].max() + linear_regression.loc[("all", "correlation")].std(),
        ],
        y=[-np.log10(0.05), -np.log10(0.05)],
        name="No correction",
        mode="lines",
        marker={"size":0.1}
)
fig.add_scatter(
        x=[
            linear_regression.loc[("all", "correlation")].min() - linear_regression.loc[("all", "correlation")].std(),
            linear_regression.loc[("all", "correlation")].max() + linear_regression.loc[("all", "correlation")].std(),
        ],
        y=[-np.log(0.05 / linear_regression.loc[("all", "sample_size")][0]), -np.log(0.05 / linear_regression.loc[("all", "sample_size")][0])],
        name="With Bonferoni correction for all",
        mode="lines",
        marker={"size":0.1}
    )

fig.show()